In [22]:
import dataset, model
import albumentations as A
import matplotlib.pyplot as plt
import pandas as pd
import torch
from torch import nn
from torch.nn import functional as F
import numpy as np
import random, os

In [23]:
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore
    
set_seed(0)

In [24]:
import importlib
importlib.reload(model)
importlib.reload(dataset)

<module 'dataset' from 'D:\\f1\\ch3\\dataset.py'>

In [4]:
import model
import albumentations as A
from albumentations.augmentations.transforms import Flip
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

if __name__ == '__main__':
    ckpt = [ModelCheckpoint(monitor = 'val_mse',save_top_k = 1,mode = 'min')
           ]
    trainer = Trainer(max_epochs = 70,gpus = 1, callbacks = ckpt, precision=32,deterministic=True,fast_dev_run = False)
    
    train_tr = A.Compose([
        #A.augmentations.transforms.ColorJitter(),
        #A.augmentations.transforms.GaussNoise()
        #A.Resize(32,100)
    ])
    
    val_tr = A.Compose([
        #A.CenterCrop(128,128),
        #A.Resize(32,100)
    ])
    
    model = model.Classifier({'lr':1e-3,'batch_size':32,'train_tr':train_tr,'val_tr':val_tr})
    
    trainer.fit(model)
    rainer.test(model)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores


In [46]:
import model
model = model.Classifier({'lr':3e-4,'batch_size':32,'train_tr':train_tr,'val_tr':val_tr})

ckpt = torch.load('lightning_logs/version_52/checkpoints/epoch=43-step=54999.ckpt') # PATH OF CHECKPOINT
model.load_state_dict(ckpt['state_dict'])
trainer.test(model)
out = trainer.predict(model)
# SHOULD SHOW A LOSS NEAR 0.002 IF NOT SOMETHING WASN'T DONE CORRECTLY

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': 0.0025073920842260122, 'test_mse': 2513.011474609375}
--------------------------------------------------------------------------------


Predicting: 0it [00:00, ?it/s]

In [47]:
def removeBatches(t):
    concat = torch.tensor([])
    for i in range(len(t)):
        concat = torch.cat((concat,torch.tensor(t[i])))
    return concat

In [48]:
import model
out = removeBatches(out)
concat = torch.mean(torch.cat([out.unsqueeze(0),out2.unsqueeze(0)],axis = 0),axis = 0)
concat = model.toNum(concat).squeeze()

In [49]:
import model
model.writeSub(concat) 

In [50]:
!aicrowd submission create -c f1-speed-recognition -f submission.csv

submission.csv ------------------- 100.0% • 105.1/103.5 KB • 1.6 MB/s • 0:00:00                                                   +-------------------------+                                                    
                                                   | Successfully submitted! |                                                    
                                                   +-------------------------+                                                    
                                                         Important links                                                          
+--------------------------------------------------------------------------------------------------------------------------------+
|  This submission | https://www.aicrowd.com/challenges/ai-blitz-8/problems/f1-speed-recognition/submissions/139839              |
|                  |                                                                                                             |
|  A